In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [3]:
sc = SparkContext()
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

In [4]:
socket_stream = ssc.socketTextStream('127.0.0.1', 9000)

In [5]:
lines = socket_stream.window(20)

In [6]:
from collections import namedtuple

In [7]:
fields = ('tag', 'count')

In [8]:
Tweet = namedtuple('Tweet', fields)

In [9]:
(lines.flatMap(lambda text: text.split(' '))
    .filter(lambda word: word.lower().startswith('#'))
    .map(lambda word: (word.lower(), 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda rec: Tweet(rec[0], rec[1]))
    .foreachRDD(lambda rdd: rdd.toDF().sort(desc('count'))
    .limit(10).registerTemplate('tweets')))

In [10]:
ssc.start()

In [11]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [12]:
count = 0
while count < 10:
    time.sleep(3)
    top_10_tweets = sqlContext.sql('Select tag, count from tweets')
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure(figsize=(10,8))
    sns.barplot(x='count', y='tag', data=top_10_df)
    sns.plt.show()
    count += 1

AnalysisException: 'Table or view not found: tweets; line 1 pos 23'

In [13]:
ssc.stop()